In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.1'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz

!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-3.0.1-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubu

In [2]:
#import packages
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType
# we are going to use this to time our queries.
import time
# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# Read in data
from pyspark import SparkFiles
peter_p_df = spark.read.text("/content/drive/MyDrive/07-PySpark/01-SkillDrill/Resources/peterpan.txt")
peter_p_df.show()

+--------------------+
|               value|
+--------------------+
|All children, exc...|
|Of course they li...|
|The way Mr. Darli...|
|Mr. Darling used ...|
|Mrs. Darling was ...|
|Wendy came first,...|
|For a week or two...|
|“Now don't interr...|
|“I have one pound...|
|“Of course we can...|
|“Remember mumps,”...|
|There was the sam...|
|Mrs. Darling love...|
|No nursery could ...|
|He had his positi...|
|Nana also trouble...|
|Mrs. Darling firs...|
|I don't know whet...|
|Of course the Nev...|
|Of all delectable...|
+--------------------+
only showing top 20 rows



In [4]:
#Reading input files and calculating word count
peter_p_data = spark.sparkContext.textFile("/content/drive/MyDrive/07-PySpark/01-SkillDrill/Resources/peterpan.txt")
peter_p_data_split = peter_p_data.flatMap(lambda line: line.split(" "))
peter_p_words_count = peter_p_data_split.map(lambda word: (word, 1))
from operator import add
peter_p_total_counts = peter_p_words_count.reduceByKey(add)

#Saving wordcount to the output file
peter_p_total_counts.saveAsTextFile("/content/drive/MyDrive/07-PySpark/01-SkillDrill/Resources/peterpanWordCounts.txt")
spark.read.text("/content/drive/MyDrive/07-PySpark/01-SkillDrill/Resources/peterpanWordCounts.txt").show()

+--------------+
|         value|
+--------------+
|  ('one,', 15)|
| ('They', 101)|
|  ('know', 64)|
|   ('way', 58)|
|  ('was', 897)|
|    ('One', 9)|
| ('when', 151)|
|   ('two', 36)|
|  ('years', 3)|
|   ('in', 623)|
|('garden,', 1)|
|('plucked', 2)|
| ('flower', 2)|
|   ('ran', 17)|
|  ('her', 361)|
|('suppose', 7)|
|  ('must', 59)|
| ('have', 243)|
|('looked', 33)|
|('rather', 40)|
+--------------+
only showing top 20 rows

